In [99]:
%matplotlib inline
from pyquil import Program, get_qc
from pyquil.gates import *
import numpy as np
import logging
import time
import matplotlib.pyplot as plt
from IPython.display import clear_output
import collections

In [100]:
def live_plot(data_dict, figsize=(7,5), title='Power Rabi'):
    clear_output(wait=True)
    plt.figure(figsize=figsize)
    for label,data in data_dict.items():
        plt.plot(data, label=label)
    plt.title(title)
    plt.grid(True)
    plt.xlabel('Pulse duration')
    plt.legend(loc='upper left') # the plot evolves to the right
    plt.show()

In [101]:
def power_rabi(qubit, number_string, frame, scale, shots=10000):
    p = Program('DECLARE ro BIT',
                f'DEFWAVEFORM mypulse:\n\t{number_string}',
                'RESET',
                f'SET-SCALE {frame} {scale}', 
                f'PULSE {frame} mypulse',
                f'MEASURE {qubit} ro'
               ).wrap_in_numshots_loop(shots)
    
    return p

In [106]:
def string_maker(length):
    ones = np.ones(length, dtype=int)
    padding_number = (4 - length % 4) % 4 # extra mod operation for length % 4 = 0
    zeros = np.zeros(padding_number, dtype=np.int8)
    
    numbers = np.concatenate((ones, zeros))
    number_string = ', '.join([str(i) for i in numbers])
    
    return number_string

In [107]:
def get_cals(qubit):
    cals = qc.compiler.get_calibration_program()
    rf_frames = list(filter(lambda f: f[0].name == "rf", cals.frames.items()))

    # save qubit frame and frequency
    (frame, frame_def) = rf_frames[qubit]
    frequency = frame_def.initial_frequency
    center = frame_def.center_frequency
    print('qubit frame: ', frame, '\tfrequency: ', frequency, '\tcenter', center)
    
    return frame, frequency, center

In [113]:
shots = 10000
qubit = 0
scale = 1

q_computer = 'Aspen-11'
qc = get_qc(q_computer)

# get calibrations and frames
frame, freq, center_freq = get_cals(qubit)

# live logging
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
    
logging.basicConfig(filename='logs/power_rabi.log', 
                    format='%(asctime)s%(msecs)03d - %(message)s', 
                    level=logging.INFO, 
                    datefmt='%d-%b-%y %H:%M:%S.')
logging.info('------------------------------------------------------------------------------   Begin')
logging.info(f'Scale: {scale}')
logging.info(f'Shots: {shots}')
logging.info(f'Qubit: {qubit}')
logging.info(f'Qubit frequency: {freq}')
logging.info(f'Qubit center frequency: {center_freq}')
logging.info(f'Quantum computer: {q_computer}')

# initialize loop
start = 1
end = 100
times = np.linspace(start, end, end - start + 1)
results = collections.defaultdict(list)

print(times)
for duration in times:
    
    # compile program and time compilation
    number_string = string_maker(int(duration))
    prog = power_rabi(qubit, number_string, frame, scale, shots)
    start_compile = time.time()
    exe = qc.compiler.native_quil_to_executable(prog)
    total_compile = time.time() - start_compile
    logging.info(f'Program compiled in {total_compile:.3f} s')
    
    # run program and get runtime
    start_run = time.time()
    # result = qc.run(exe)
    result = qc.run(exe).readout_data.get('ro')
    total_run = time.time() - start_run
    
    # update plot and log
    probability = np.sum(result) / shots
    results['Probability'].append(probability)
    live_plot(results)
    logging.info(f' duration: {duration} \t-- probability: {probability} \t--- runtime: {total_run:.3f} seconds')

KeyboardInterrupt: 